# Example PyMongo Snippets

- **connect to cluster and get reference to a MongoDB collection**

In [ ]:
from pymongo_get_database import get_database


db = get_database( db_name="sample_mflix" )

display( f"connected to MongoDB cluster: {db.name}" )


## Use the Aggregation Framework to query for "A Star Is Born"

In [ ]:
import pymongo 
from IPython.core.display import Markdown as md
from prettytable import PrettyTable

# Get a reference to the "movies" collection:
movie_collection = db["movies"]

# Match title = "A Star Is Born":
stage_match_title = {
   "$match": {
         "title": "A Star Is Born"
   }
}

# Sort by year, ascending:
stage_sort_year_ascending = {
   "$sort": { "year": pymongo.ASCENDING }
}

# Sort by year, ascending:
stage_sort_year_descending = {
   "$sort": { "year": pymongo.DESCENDING }
}

# Now the pipeline is easier to read:
pipeline = [
    stage_match_title, 
    stage_sort_year_ascending,
   #  stage_project
]

# make it pretty
results = list(movie_collection.aggregate(pipeline))
table = PrettyTable()
for movie in results:
   table.add_row([movie["title"], movie["year"], ', '.join( movie["cast"])])
table.format = True
display( md(table.get_html_string(attributes={'align': 'left', 'class': 'table table-striped'})) )

### Bucket/Window Example


In [ ]:
stage_unwind = {
   "$unwind": {
       "path": "$genres"
   }
}

stage_addFields = {
   "$addFields": {
       "decade": {
         "$multiply": [{ "$floor": { "$divide": ["$year", 10 ] }}, 10 ]
       }
   }
}

stage_group = {
   "$group": {
       "_id": {
           "decade": { "$toInt": "$decade" }, "genre": "$genres"
       },
       "count": {
           "$count": {}
       }
   }
}

stage_project = {
   "$project": {
       "count": 1,
       "_id": 1
   }
}

stage_sort = {
    "$sort": {"_id.decade":-1, "count":-1 }
}

pipeline = [ stage_sort_year_descending, stage_unwind, stage_addFields, stage_group, stage_sort ]

result = list(movie_collection.aggregate(pipeline))
display(result)

## Transactions

In [ ]:
from time import sleep
from pymongo.errors import DuplicateKeyError

with db.client.start_session() as session:
    with session.start_transaction():
        _id = 1
        while True:
            try:
                doc = { "_id" : _id, "x": 1 }
                display( f"Inserting doc {doc}")
                display( db.test.insert_one( doc ) )
                break
            except DuplicateKeyError:
                display( f"_id {_id} already exists" )
                _id += 1
            except Exception as e:
                session.abort_transaction()
                display( f"Transaction aborted: {e}")
                break

display( list(db.test.find()) )


- **Delete test records**

In [ ]:
with db.client.start_session() as session:
    with session.start_transaction():
        display( db.test.delete_many({ "x": 1 }) )

display( f"Finding deleted doc: {db.test.find_one()}" )

## PyMongoArrow

In [ ]:
from pyarrow import list_, string, int32
from pandas import DataFrame
from pymongoarrow.monkey import patch_all
patch_all()
from pymongoarrow.api import Schema
import numpy

schema = Schema( { "title": str, "metacritic": float, "genres": list_(string()) } )
# df = db.movies.find_pandas_all( { "directors": "David Lynch" }, schema=schema )
arrow = db.movies.find_arrow_all( { "directors": "David Lynch" }, schema=schema )
df = arrow.to_pandas().sort_values( by="metacritic", ascending=False )

display( md( df.to_html() ) )

display( df.agg( {"metacritic": ["mean", "min", "max"]} ) )
